In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import matplotlib.pyplot as plt

In [2]:
# --- CARGAR EL DATASET ---
df = pd.read_csv("limpieza_y_extraccion_de_datos/datasets/startups_para_modelar.csv", sep=",")

In [3]:
# --- NORMALIZACIÓN Y CODIFICACIÓN DE VARIABLES ---

# Variables para transformación logarítmica
log_vars = [
    'financiacion_total_por_acciones', 'ultima_financiacion_por_acciones',
    'monto_de_ultima_financiacion', 'financiacion_total',
    'seguidores_en_linkedin', 'seguidores_en_facebook', 'seguidores_en_twitter',
    'empleados_en_linkedin'
    
    , 'años_desde_ultima_financiacion'
]

# Asegurar tipos numéricos
df[log_vars] = df[log_vars].apply(pd.to_numeric, errors='coerce')

# Crear columnas logarítmicas
for col in log_vars:
    df[f"{col}_log"] = np.log1p(df[col])

# Variables numéricas
num_vars = [
    'numero_de_marcas_registradas', 'numero_de_patentes_otorgadas',
    'numero_total_de_fundadores', 'numero_de_mujeres_fundadores',
    'numero_de_hombres_fundadores', 'numero_de_inversionistas',
    'numero_de_inversionistas_principales', 'numero_de_inversiones_en_rondas_de_financiacion',
    'numero_de_articulos', 'numero_de_rondas_de_financiacion',
    'antiguedad_de_la_empresa'
] + [f"{col}_log" for col in log_vars]

# Variables categóricas
cat_vars = ['categoria', 'tipo_de_ultima_financiacion' 
           
           ]

# Preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_vars),
       ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_vars)
    ]
)



In [4]:
#--- TRANSFORMACIÓN ---
X = preprocessor.fit_transform(df)

In [5]:
# --- DETERMINACIÓN DEL NÚMERO ÓPTIMO DE CLÚSTERES ---

# Rango de K a probar
K_range = range(2, 11)  # desde 2 hasta 10 clústeres

# Listas para almacenar métricas
inertias = []
silhouettes = []
db_scores = []
ch_scores = []
inertia = []


# Calcular métricas para cada K
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    
    silhouettes.append(silhouette_score(X, labels))
    db_scores.append(davies_bouldin_score(X, labels))
    ch_scores.append(calinski_harabasz_score(X, labels))
    inertia.append(kmeans.inertia_)

# --- Gráficas ---
fig, axs = plt.subplots(1, 3, figsize=(20, 7))
fig.suptitle("Selección del Número Óptimo de Clústeres (K)", fontsize=16)


# Silhouette
axs[0].plot(K_range, silhouettes, marker='o', color='green')
axs[0].set_title('Silhouette Score')
axs[0].set_xlabel('K')
#axs[1].set_ylabel('Silhouette')

# Davies-Bouldin
axs[1].plot(K_range, db_scores, marker='o', color='red')
axs[1].set_title('Davies-Bouldin Index')
axs[1].set_xlabel('K')
#axs[1].set_ylabel('Davies-Bouldin')

# Calinski-Harabasz 
axs[2].plot(K_range, ch_scores, marker='o', color='red')
axs[2].set_title('Calinski-Harabasz  Index')
axs[2].set_xlabel('K')
#axs[2].set_ylabel('Calinski-Harabasz ')


plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# --- Sugerencias ---
best_k_silhouette = K_range[silhouettes.index(max(silhouettes))]
best_k_davies = K_range[db_scores.index(min(db_scores))]
best_k_ch = K_range[ch_scores.index(max(ch_scores))]


print("🔍 Sugerencias basadas en métricas:")
print(f"→ Mejor K por Silhouette Score: {best_k_silhouette}")
print(f"→ Mejor K por Davies-Bouldin Index: {best_k_davies}")
print(f"→ Mejor K por Calinski-Harabasz Index: {best_k_ch}")


NameError: name 'KMeans' is not defined

In [ ]:
# --- SE GENERAN GRÁFICAS PARA EL TRABAJO ESCRITO --- 

# Crear carpeta si no existe
output_dir = "mejor_K"
os.makedirs(output_dir, exist_ok=True)

# --- Silhouette ---
plt.figure(figsize=(8, 5))
plt.plot(K_range, silhouettes, marker='o', color='green')
plt.axvline(best_k_silhouette, color='blue', linestyle='--', label=f'Mejor K = {best_k_silhouette}')
#plt.title('Silhouette Score')
plt.xlabel('K')
#plt.ylabel('Silhouette')
plt.legend()
plt.grid(True)

# Guardar imagen
path = os.path.join(output_dir, "silhouette_score.png")
plt.savefig(path, dpi=300, bbox_inches='tight')

plt.show()

# --- Davies-Bouldin ---
plt.figure(figsize=(8, 5))
plt.plot(K_range, db_scores, marker='o', color='red')
plt.axvline(best_k_davies, color='blue', linestyle='--', label=f'Mejor K = {best_k_davies}')
#plt.title('Davies-Bouldin Index')
plt.xlabel('K')
#plt.ylabel('Davies-Bouldin')
plt.legend()
plt.grid(True)

# Guardar imagen
path = os.path.join(output_dir, "davies_bouldin_index.png")
plt.savefig(path, dpi=300, bbox_inches='tight')

plt.show()

# --- Calinski-Harabasz ---
plt.figure(figsize=(8, 5))
plt.plot(K_range, ch_scores, marker='o', color='purple')
plt.axvline(best_k_ch, color='blue', linestyle='--', label=f'Mejor K = {best_k_ch}')
#plt.title('Calinski-Harabasz Index')
plt.xlabel('K')
#plt.ylabel('Calinski-Harabasz')
plt.legend()
plt.grid(True)

# Guardar imagen
path = os.path.join(output_dir, "calinski_harabasz_index.png")
plt.savefig(path, dpi=300, bbox_inches='tight')

plt.show()